In [1]:
# boilerplate code
from __future__ import print_function
import os
from io import BytesIO
import numpy as np
from functools import partial
import PIL.Image
from IPython.display import clear_output, Image, display, HTML

import tensorflow as tf

C:\Users\jdu12\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#!pip install --quiet lucid==0.0.5
#!npm install -g svelte-cli@2.2.0

In [3]:
## print the layers
graph_file = 'C:/Users/jdu12/Desktop/research/saved_models/GoogleNet_rect/rect.pb'
graph_def = tf.GraphDef()
with open(graph_file, "rb") as f:
    graph_def.ParseFromString(f.read())
for node in graph_def.node:
    print(node.name)

input_1
Conv2d_1a_7x7_conv/kernel
Conv2d_1a_7x7_conv/kernel/read
Conv2d_1a_7x7_conv/convolution
Conv2d_1a_7x7_bn/beta
Conv2d_1a_7x7_bn/beta/read
Conv2d_1a_7x7_bn/moving_mean
Conv2d_1a_7x7_bn/moving_mean/read
Conv2d_1a_7x7_bn/moving_variance
Conv2d_1a_7x7_bn/moving_variance/read
Conv2d_1a_7x7_bn/batchnorm/add/y
Conv2d_1a_7x7_bn/batchnorm/add
Conv2d_1a_7x7_bn/batchnorm/Rsqrt
Conv2d_1a_7x7_bn/batchnorm/mul
Conv2d_1a_7x7_bn/batchnorm/mul_1
Conv2d_1a_7x7_bn/batchnorm/sub
Conv2d_1a_7x7_bn/batchnorm/add_1
Conv2d_1a_7x7_act/Relu
MaxPool_2a_3x3/MaxPool
Conv2d_2b_1x1_conv/kernel
Conv2d_2b_1x1_conv/kernel/read
Conv2d_2b_1x1_conv/convolution
Conv2d_2b_1x1_bn/beta
Conv2d_2b_1x1_bn/beta/read
Conv2d_2b_1x1_bn/moving_mean
Conv2d_2b_1x1_bn/moving_mean/read
Conv2d_2b_1x1_bn/moving_variance
Conv2d_2b_1x1_bn/moving_variance/read
Conv2d_2b_1x1_bn/batchnorm/add/y
Conv2d_2b_1x1_bn/batchnorm/add
Conv2d_2b_1x1_bn/batchnorm/Rsqrt
Conv2d_2b_1x1_bn/batchnorm/mul
Conv2d_2b_1x1_bn/batchnorm/mul_1
Conv2d_2b_1x1_bn/b

In [4]:
model_fn = 'C:/Users/jdu12/Desktop/research/saved_models/GoogleNet_rect/rect.pb'

# creating TensorFlow session and loading the model
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)
with tf.gfile.FastGFile(model_fn, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
t_input = tf.placeholder(np.float32, name='input_1') # define the input tensor
imagenet_mean = 0.0
t_preprocessed = tf.expand_dims(t_input-imagenet_mean, 0)
tf.import_graph_def(graph_def, {'input_1':t_preprocessed})

In [5]:
layers = [op.name for op in graph.get_operations() if op.type=='Conv2D' and 'import/' in op.name]
feature_nums = [int(graph.get_tensor_by_name(name+':0').get_shape()[-1]) for name in layers]

print('Number of layers', len(layers))
print('Total number of feature channels:', sum(feature_nums))


# Helper functions for TF Graph visualization

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

# Visualizing the network graph. Be sure expand the "mixed" nodes to see their 
# internal structure. We are going to visualize "Conv2D" nodes.
tmp_def = rename_nodes(graph_def, lambda s:"/".join(s.split('_',1)))
show_graph(tmp_def)

Number of layers 58
Total number of feature channels: 7316


In [6]:
'''
import lucid.optvis.render as render
from lucid.modelzoo.vision_base import Model

class InceptionV1(Model):
    model_path = 'C:/Users/jdu12/Desktop/research/saved_models/GoogleNet_rect/rect.pb'
    labels = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35']
    image_shape = [224, 224, 3]
    image_value_range = (0, 1)
    input_name = 'input_1'
'''

"\nimport lucid.optvis.render as render\nfrom lucid.modelzoo.vision_base import Model\n\nclass InceptionV1(Model):\n    model_path = 'C:/Users/jdu12/Desktop/research/saved_models/GoogleNet_rect/rect.pb'\n    labels = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35']\n    image_shape = [224, 224, 3]\n    image_value_range = (0, 1)\n    input_name = 'input_1'\n"

In [7]:

'''
model = InceptionV1()
model.load_graphdef()

clear_output()
#_ = render.render_vis(model, "Mixed_4e_Concatenated/concat:1")
#_ = render.render_vis(model, "Mixed_4f_Branch_4_b_1x1_conv/convolution:20")

four_e0 = render.render_vis(model, "Mixed_4e_Concatenated/concat:0")
#four_e1 = render.render_vis(model, "Mixed_4e_Concatenated/concat:1")
#_ = render.render_vis(model, "Mixed_4e_Concatenated/concat:2")
#_ = render.render_vis(model, "Mixed_4e_Concatenated/concat:3")
#_ = render.render_vis(model, "Mixed_4e_Concatenated/concat:4")
#_ = render.render_vis(model, "Mixed_4e_Concatenated/concat:5")
'''

'\nmodel = InceptionV1()\nmodel.load_graphdef()\n\nclear_output()\n#_ = render.render_vis(model, "Mixed_4e_Concatenated/concat:1")\n#_ = render.render_vis(model, "Mixed_4f_Branch_4_b_1x1_conv/convolution:20")\n\nfour_e0 = render.render_vis(model, "Mixed_4e_Concatenated/concat:0")\n#four_e1 = render.render_vis(model, "Mixed_4e_Concatenated/concat:1")\n#_ = render.render_vis(model, "Mixed_4e_Concatenated/concat:2")\n#_ = render.render_vis(model, "Mixed_4e_Concatenated/concat:3")\n#_ = render.render_vis(model, "Mixed_4e_Concatenated/concat:4")\n#_ = render.render_vis(model, "Mixed_4e_Concatenated/concat:5")\n'

# Naive feature visualization
Let's start with a naive way of visualizing these. Image-space gradient ascent!

In [8]:
# Picking some internal layer. Note that we use outputs before applying the ReLU nonlinearity
# to have non-zero gradients for features with negative initial activations.
layer = 'Mixed_4d_Concatenated/concat'
channel = 139 # picking some feature channel to visualize

# start with a gray image with a little noise
img_noise = np.random.uniform(size=(224,224,3)) + 100.0

def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 1)*255)
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
    
def visstd(a, s=0.1):
    '''Normalize the image range for visualization'''
    return (a-a.mean())/max(a.std(), 1e-4)*s + 0.5

def T(layer):
    '''Helper for getting layer output tensor'''
    return graph.get_tensor_by_name("import/%s:0"%layer)

def render_naive(t_obj, img0=img_noise, iter_n=20, step=1.0):
    t_score = tf.reduce_mean(t_obj) # defining the optimization objective
    t_grad = tf.gradients(t_score, t_input)[0] # behold the power of automatic differentiation!
    
    img = img0.copy()
    for i in range(iter_n):
        g, score = sess.run([t_grad, t_score], {t_input:img})
        # normalizing the gradient, so the same step size should work 
        g /= g.std()+1e-8         # for different layers and networks
        img += g*step
        print(score, end = ' ')
    clear_output()
    showarray(visstd(img))

#render_naive(T(layer)[:,:,:,channel])

# Multiscale image generation
Looks like the network wants to show us something interesting! Let's help it. We are going to apply gradient ascent on multiple scales. Details formed on smaller scale will be upscaled and augmented with additional details on the next scale.
With multiscale image generation it may be tempting to set the number of octaves to some high value to produce wallpaper-sized images. Storing network activations and backprop values will quickly run out of GPU memory in this case. There is a simple trick to avoid this: split the image into smaller tiles and compute each tile gradient independently. Applying random shifts to the image before every iteration helps avoid tile seams and improves the overall image quality.

In [9]:
def tffunc(*argtypes):
    '''Helper that transforms TF-graph generating function into a regular one.
    See "resize" function below.
    '''
    placeholders = list(map(tf.placeholder, argtypes))
    def wrap(f):
        out = f(*placeholders)
        def wrapper(*args, **kw):
            return out.eval(dict(zip(placeholders, args)), session=kw.get('session'))
        return wrapper
    return wrap

# Helper function that uses TF to resize an image
def resize(img, size):
    img = tf.expand_dims(img, 0)
    return tf.image.resize_bilinear(img, size)[0,:,:,:]
resize = tffunc(np.float32, np.int32)(resize)


def calc_grad_tiled(img, t_grad, tile_size=512):
    '''Compute the value of tensor t_grad over the image in a tiled way.
    Random shifts are applied to the image to blur tile boundaries over 
    multiple iterations.'''
    sz = tile_size
    h, w = img.shape[:2]
    sx, sy = np.random.randint(sz, size=2)
    img_shift = np.roll(np.roll(img, sx, 1), sy, 0)
    grad = np.zeros_like(img)
    for y in range(0, max(h-sz//2, sz),sz):
        for x in range(0, max(w-sz//2, sz),sz):
            sub = img_shift[y:y+sz,x:x+sz]
            g = sess.run(t_grad, {t_input:sub})
            grad[y:y+sz,x:x+sz] = g
    return np.roll(np.roll(grad, -sx, 1), -sy, 0)

In [10]:
def render_multiscale(t_obj, img0=img_noise, iter_n=10, step=1.0, octave_n=3, octave_scale=1.4):
    t_score = tf.reduce_mean(t_obj) # defining the optimization objective
    t_grad = tf.gradients(t_score, t_input)[0] # behold the power of automatic differentiation!
    
    img = img0.copy()
    for octave in range(octave_n):
        if octave>0:
            hw = np.float32(img.shape[:2])*octave_scale
            img = resize(img, np.int32(hw))
        for i in range(iter_n):
            g = calc_grad_tiled(img, t_grad)
            # normalizing the gradient, so the same step size should work 
            g /= g.std()+1e-8         # for different layers and networks
            img += g*step
            print('.', end = ' ')
        clear_output()
        showarray(visstd(img))

#render_multiscale(T(layer)[:,:,:,channel])

# Laplacian Pyramid Gradient Normalization
This looks better, but the resulting images mostly contain high frequencies. Can we improve it? One way is to add a smoothness prior into the optimization objective. This will effectively blur the image a little every iteration, suppressing the higher frequencies, so that the lower frequencies can catch up. This will require more iterations to produce a nice image. Why don't we just boost lower frequencies of the gradient instead? One way to achieve this is through the Laplacian pyramid decomposition. We call the resulting technique Laplacian Pyramid Gradient Normalization.

In [11]:
k = np.float32([1,4,6,4,1])
k = np.outer(k, k)
k5x5 = k[:,:,None,None]/k.sum()*np.eye(3, dtype=np.float32)

def lap_split(img):
    '''Split the image into lo and hi frequency components'''
    with tf.name_scope('split'):
        lo = tf.nn.conv2d(img, k5x5, [1,2,2,1], 'SAME')
        lo2 = tf.nn.conv2d_transpose(lo, k5x5*4, tf.shape(img), [1,2,2,1])
        hi = img-lo2
    return lo, hi

def lap_split_n(img, n):
    '''Build Laplacian pyramid with n splits'''
    levels = []
    for i in range(n):
        img, hi = lap_split(img)
        levels.append(hi)
    levels.append(img)
    return levels[::-1]

def lap_merge(levels):
    '''Merge Laplacian pyramid'''
    img = levels[0]
    for hi in levels[1:]:
        with tf.name_scope('merge'):
            img = tf.nn.conv2d_transpose(img, k5x5*4, tf.shape(hi), [1,2,2,1]) + hi
    return img

def normalize_std(img, eps=1e-10):
    '''Normalize image by making its standard deviation = 1.0'''
    with tf.name_scope('normalize'):
        std = tf.sqrt(tf.reduce_mean(tf.square(img)))
        return img/tf.maximum(std, eps)

def lap_normalize(img, scale_n=4):
    '''Perform the Laplacian pyramid normalization.'''
    img = tf.expand_dims(img,0)
    tlevels = lap_split_n(img, scale_n)
    tlevels = list(map(normalize_std, tlevels))
    out = lap_merge(tlevels)
    return out[0,:,:,:]

# Showing the lap_normalize graph with TensorBoard
lap_graph = tf.Graph()
with lap_graph.as_default():
    lap_in = tf.placeholder(np.float32, name='lap_in')
    lap_out = lap_normalize(lap_in)
show_graph(lap_graph)

In [12]:
def savearray(a, save_dir, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 1)*255)
    f = BytesIO()
    PIL.Image.fromarray(a).save(save_dir, fmt)
    clear_output()
    #display(Image(data=save_dir.getvalue()))
    print(save_dir)

def render_lapnorm(t_obj, save_dir, img0=img_noise, visfunc=visstd,
                   iter_n=10, step=1.0, octave_n=3, octave_scale=1.4, lap_n=4):
    t_score = tf.reduce_mean(t_obj) # defining the optimization objective
    t_grad = tf.gradients(t_score, t_input)[0] # behold the power of automatic differentiation!
    # build the laplacian normalization graph
    lap_norm_func = tffunc(np.float32)(partial(lap_normalize, scale_n=lap_n))

    img = img0.copy()
    for octave in range(octave_n):
        if octave>0:
            hw = np.float32(img.shape[:2])*octave_scale
            img = resize(img, np.int32(hw))
        for i in range(iter_n):
            g = calc_grad_tiled(img, t_grad)
            g = lap_norm_func(g)
            img += g*step
            print('.', end = ' ')
        clear_output()
        #showarray(visfunc(img))
        #print(type(img))
        #print(img.shape)
    savearray(visfunc(img), save_dir)


In [13]:
root_dir = 'C:/Users/jdu12/Desktop/research/sprite/'
filter_num = [256, 480, 512, 512, 512, 528, 832, 832, 1024]
folder_dir = ['3b/', '3c/', '4b/', '4c/', '4d/', '4e/', '4f/', '5b/', '5c/']
layers = [
    'Mixed_3b_Concatenated/concat',
    'Mixed_3c_Concatenated/concat',
    'Mixed_4b_Concatenated/concat',
    'Mixed_4c_Concatenated/concat',
    'Mixed_4d_Concatenated/concat',
    'Mixed_4e_Concatenated/concat',
    'Mixed_4f_Concatenated/concat',
    'Mixed_5b_Concatenated/concat',
    'Mixed_5c_Concatenated/concat'
]
#for i in range(9):    # 9 layers
i = 7
cur_filter_num = filter_num[i]
cur_folder_dir = folder_dir[i]
cur_layer = layers[i]
for j in range(765, cur_filter_num):
    save_dir = root_dir + cur_folder_dir + str(j) + '.jpeg'
    # j is the current channel we are doing
    render_lapnorm(T(cur_layer)[:,:,:,j], save_dir)

print('layer', folder_dir[i], 'is done !!!!!!!!!!!!!!!!!!!!')

C:/Users/jdu12/Desktop/research/sprite/5b/831.jpeg
layer 5b/ is done !!!!!!!!!!!!!!!!!!!!
